# Tutorial for exploring the occupational and skills data

Here, we show how to access and explore:
1. Occupation profiles
2. Occupational groups (ISCO)
3. Skills-based sectors and sub-sectors
4. Skills and skills categories
5. Occupation similarities

In [1]:
import mapping_career_causeways.load_data_utils as load_data

Module `load_data_utils` features two classes: `Data` can be used to easily load in occupational and skills data, and `Similarities` can be used to access similarity matrices.

In [2]:
data = load_data.Data()

# 1. Occupation profiles

One of the main tables is the occupational profiles table `occ`, that contains various descriptors that we used to find career transitions that are *viable* (a good fit in terms of skills, work activities, work context, and education and experience levels), *desirable* (in terms of earnings), and *safe* (in terms of automation risk). 

The descriptions for each of the fields can be found in the [readme on the project's githbub repository](#).

In [3]:
data.occ.head(3)

,id,concept_type,concept_uri,preferred_label,isco_level_1,isco_level_2,isco_level_3,isco_level_4,is_top_level,job_zone,...,onet_occupation,level_1,level_2,skills_based_sector_code,sub_sector_code,skills_based_sector,sub_sector,remote_labor_index,physical_proximity,exposure_score
0,0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,technical director,2,21,216,2166,False,4.0,...,art directors,5,25,6,6.3,arts & media workers,creative managers & graphic designers,0.733333,0.5850,0.394968
1,1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,metal drawing machine operator,8,81,812,8121,True,2.0,...,"extruding and drawing machine setters, operato...",3,16,4,4.1,manufacturing workers,metal workers & machine operators,0.125000,0.5125,0.669655
2,2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,precision device inspector,7,75,754,7543,False,2.0,...,"inspectors, testers, sorters, samplers, and we...",0,0,1,1.1,"technicians, construction, transport & securit...",engineering technicians,0.095238,0.6450,0.763918


In [4]:
# Note that data.occ contains profiles for all 2942 ESCO occupations
data.occ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2942 entries, 0 to 2941
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        2942 non-null   int64  
 1   concept_type              2942 non-null   object 
 2   concept_uri               2942 non-null   object 
 3   preferred_label           2942 non-null   object 
 4   isco_level_1              2942 non-null   int64  
 5   isco_level_2              2942 non-null   int64  
 6   isco_level_3              2942 non-null   int64  
 7   isco_level_4              2942 non-null   int64  
 8   is_top_level              2942 non-null   bool   
 9   job_zone                  2860 non-null   float64
 10  education_level           2858 non-null   float64
 11  related_work_experience   2858 non-null   float64
 12  on_the_job_training       2858 non-null   float64
 13  annual_earnings           2926 non-null   float64
 14  total_pa

Note that each occupation also has an official one or few sentence description written by ESCO, which is stored separately (as it is not often used) and can be accessed via `data.occupations.description`

In [5]:
data.occupations.loc[0].description

'Technical directors realise the artistic visions of the creators within technical constraints. They coordinate the operations of various production units, such as scene, wardrobe, sound and lighting, and make-up. They adapt the prototype and study the feasibility, implementation, operation and technical monitoring of the artistic project. They are also responsible for the stage equipment and technical equipment.'

You might want to find the profile of a specific occupation - to find the ID number for this occupation given the occupation title, you can use `occ_title_to_id()`. Note that presently you have to provide an exact match of the occupation's preferred label to find the ID number. To find the title that you're interested in, you can easily browse ESCO occupations in the [ESCO portal](https://ec.europa.eu/esco/portal/occupation).

In [6]:
data.occ_title_to_id('precision device inspector')

2

After generating transition recommendations, you might wish to supplement the table of transitions with additional information from the occupational profile table. This can be easily done using `add_field_to_occupation()`.

Below, we generate safe and desirable transitions for the 'shop assistant', and add information about what fraction of destination occupation's tasks can be performed while working from home (using the `remote_labor_index` field).

In [7]:
import mapping_career_causeways.transitions_utils as trans_utils

# Generate transitions
transitions = trans_utils.get_transitions(origin_ids=[data.occ_title_to_id('shop assistant')], less_information=True)
transitions = transitions[transitions['is_safe_desirable']]

# Add remote_labor_index
df = data.add_field_to_occupation(
    transitions, # dataframe to which to add the field
    occupation_id_col = 'destination_id', #  which column hosts the occupation IDs
    field_name = 'remote_labor_index', # field to add
    new_prefix = 'destination' # prefix to add to the field name
).sort_values('destination_remote_labor_index', ascending=False).dropna()

In [8]:
# Occupations where most of the activities can be performed remotely
df[['origin_label', 'destination_label', 'similarity', 'destination_remote_labor_index']].head(5)

,origin_label,destination_label,similarity,destination_remote_labor_index
5,shop assistant,customer contact centre information clerk,0.379332,0.695652
20,shop assistant,customer service representative,0.305846,0.695652
14,shop assistant,client relations manager,0.320825,0.695652
12,shop assistant,ICT help desk agent,0.324065,0.647059
15,shop assistant,sales processor,0.320485,0.600000


In [9]:
# Occupations where most of the activities must be performed locally
df[['origin_label', 'destination_label', 'similarity', 'destination_remote_labor_index']].tail(5)

,origin_label,destination_label,similarity,destination_remote_labor_index
9,shop assistant,cloak room attendant,0.334423,0.285714
11,shop assistant,beauty salon attendant,0.327997,0.285714
18,shop assistant,railway passenger service agent,0.309568,0.250000
22,shop assistant,parking valet,0.303920,0.214286
13,shop assistant,tanning consultant,0.322397,0.100000


# 2. Occupational groups

You can also select occupations based on their ISCO group. To access the different ISCO-08 codes and their descriptions, you can use the `data.isco_titles` table.

In [10]:
# Find the code for ISCO unit group 'Shop sales assistants'
data.isco_titles[data.isco_titles.isco_title=='Shop sales assistants']

,isco,isco_title,level
433,5223,Shop sales assistants,4


In [11]:
# Find all 'top level' occupations in the shop sales assistants group
data.occ_report[data.occ_report.isco_level_4==5223]

,id,concept_type,concept_uri,preferred_label,isco_level_1,isco_level_2,isco_level_3,isco_level_4,is_top_level,job_zone,...,onet_occupation,level_1,level_2,skills_based_sector_code,sub_sector_code,skills_based_sector,sub_sector,remote_labor_index,physical_proximity,exposure_score
66,119,Occupation,http://data.europa.eu/esco/occupation/099a901f...,motor vehicles parts advisor,5,52,522,5223,True,2.0,...,parts salespersons,2,12,3,3.2,sales & services workers,customer representatives,0.571429,0.6725,0.536856
77,139,Occupation,http://data.europa.eu/esco/occupation/0b15375e...,shop assistant,5,52,522,5223,True,2.0,...,retail salespersons,2,14,3,3.4,sales & services workers,retail & sales workers,0.631579,0.6850,0.502363
374,681,Occupation,http://data.europa.eu/esco/occupation/38395ab3...,car leasing agent,5,52,522,5223,True,2.0,...,retail salespersons,2,12,3,3.2,sales & services workers,customer representatives,0.631579,0.6850,0.502363
549,1010,Occupation,http://data.europa.eu/esco/occupation/547b304b...,specialised seller,5,52,522,5223,True,2.0,...,retail salespersons,2,14,3,3.4,sales & services workers,retail & sales workers,0.631579,0.6850,0.502363
766,1380,Occupation,http://data.europa.eu/esco/occupation/73357956...,sales processor,5,52,522,5223,True,2.0,...,order clerks,2,12,3,3.2,sales & services workers,customer representatives,0.600000,0.5550,0.471169
1046,1862,Occupation,http://data.europa.eu/esco/occupation/9ba74e8a...,sales assistant,5,52,522,5223,True,2.0,...,retail salespersons,2,14,3,3.4,sales & services workers,retail & sales workers,0.631579,0.6850,0.502363
1585,2862,Occupation,http://data.europa.eu/esco/occupation/f8da6546...,personal shopper,5,52,522,5223,True,2.0,...,retail salespersons,2,14,3,3.4,sales & services workers,retail & sales workers,0.631579,0.6850,0.502363


Note that in the cell above we used `data.occ_report` which features a restricted subset of profiles, featuring the 1627 ESCO occupations that were analysed in the Mapping Career Causeways report. These occupations are both the so-called top level ESCO occupations (they can be seen as fifth-digit occupations of the ISCO classification) and they have estimates of automation risk.

For the full list of the top level ESCO occupations, see `data.occ_top` (includes occupations for which we did not estimate the automation risk).

# 3. Skills-based sectors and sub-sectors
Groups of related occupations can also be assessed by using skills-based sectors and sub-sectors. We found these sectors by using graph-based clustering and they group together occupations that have similar skills sets, work activities and work context features. As such, it is not an official hierarchy of occupations, and it should rather be seen as an experimental classification (for more details, see the notebook `Clustering_01_ESCO_occupations.ipynb` in `05_transition_analyses/`, as well as the appendix of the Mapping Career Causeways report).

In [12]:
# All skills-based sectors and their codes
data.skills_based_sectors

,skills_based_sector_code,skills_based_sector,level_1
0,1,"technicians, construction, transport & securit...",0
1,2,business & administration workers,1
2,3,sales & services workers,2
3,4,manufacturing workers,3
4,5,engineers & researchers,4
5,6,arts & media workers,5
6,7,education workers,6
7,8,"textile, clothing, leather & footwear manufact...",7
8,9,food & tobacco production workers,8
9,10,logistics workers,9


As an example, let's select roles in the *customer representatives* sub-sector, which is part of the *sales and services* skills-based sector. Note that the columns `{skills_based/sub}_sector_code` and `level_{1/2}` are simply two alternative codings of the clusters.

In [13]:
# All sub-sectors within 'sales and services'
data.sub_sectors[data.sub_sectors.level_1==2]

,sub_sector_code,sub_sector,skills_based_sector_code,skills_based_sector,level_1,level_2
11,3.1,"food, cleaning & services workers",3,sales & services workers,2,11
12,3.2,customer representatives,3,sales & services workers,2,12
13,3.3,shop & services managers,3,sales & services workers,2,13
14,3.4,retail & sales workers,3,sales & services workers,2,14
15,3.5,wholesale merchants,3,sales & services workers,2,15


In [14]:
# Roles in the 'customer representatives' sub-sector
data.occ_report[data.occ_report.sub_sector=='customer representatives'].head(3)

,id,concept_type,concept_uri,preferred_label,isco_level_1,isco_level_2,isco_level_3,isco_level_4,is_top_level,job_zone,...,onet_occupation,level_1,level_2,skills_based_sector_code,sub_sector_code,skills_based_sector,sub_sector,remote_labor_index,physical_proximity,exposure_score
66,119,Occupation,http://data.europa.eu/esco/occupation/099a901f...,motor vehicles parts advisor,5,52,522,5223,True,2.0,...,parts salespersons,2,12,3,3.2,sales & services workers,customer representatives,0.571429,0.6725,0.536856
107,186,Occupation,http://data.europa.eu/esco/occupation/0ededdc2...,call centre agent,5,52,524,5244,True,2.0,...,telemarketers,2,12,3,3.2,sales & services workers,customer representatives,0.750000,0.7675,0.438035
125,215,Occupation,http://data.europa.eu/esco/occupation/114e1eff...,renewable energy consultant,2,24,243,2433,True,4.0,...,sustainability specialists,2,12,3,3.2,sales & services workers,customer representatives,0.703704,0.4650,0.371184


# 4. Skills

Table `skills` provides information about the 13,000+ [ESCO skills items](https://ec.europa.eu/esco/portal/skill) and their place in the ESCO hierarchy.

In [15]:
data.skills.head(3)

,concept_type,concept_uri,skill_type,reuse_level,preferred_label,alt_labels,description,id,code,skill_category,level_1,level_2,level_3,level_title,level_href,level_uri
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,manage musical staff,manage staff of music\ncoordinate duties of mu...,Assign and manage staff tasks in areas such as...,0,S4.8.1,S,S4,S4.8,S4.8.1,supervising a team or group,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S4.8.1
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,supervise correctional procedures,oversee prison procedures\nmanage correctional...,Supervise the operations of a correctional fac...,1,S4.2.1,S,S4,S4.2,S4.2.1,directing operational activities,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S4.2.1
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,apply anti-oppressive practices,apply non-oppressive practices\napply an anti-...,"Identify oppression in societies, economies, c...",2,S3.3.0,S,S3,S3.3,S3.3.0,protecting and enforcing,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S3.3.0


The information about specific hierarchical categories and their relationships can be found in `data.concepts`; the data on skills hierarchy was obtained via ESCO API (see `ESCO_skills_hierarchy.ipynb` in `00_preprocessing/`).

In [16]:
data.concepts[data.concepts.code=='S4.8.1']

,title,category,code,description_en,hierarchy_level,broader_concept_code,uri,broader_concept_uri,number_of_items
483,supervising a team or group,S,S4.8.1,"Managing, leading or supervising the activitie...",3,S4.8,http://data.europa.eu/esco/skill/S4.8.1,http://data.europa.eu/esco/skill/S4.8,79


To find skills for a specific occupation, you can use `data.occupation_skills()` function. Note that ESCO distinguishes between 'Essential' and 'Optional' skills. We use this distinction to calculate two different sets of occupation similarity measures.

In [17]:
# Find all skills related to the shop assistant occupation
data.occupation_skills(
    data.occ_title_to_id('hotel concierge'), # occupation's ID number
    skill_importance=None # set to None, 'Essential' or 'Optional'
)

,occupation_id,skill_id,importance,preferred_label,description
0,329,2052,Essential,greet guests,Welcome guests in a friendly manner in a certa...
1,329,5558,Essential,identify customer's needs,Use appropriate questions and active listening...
2,329,6288,Essential,maintain customer service,Keep the highest possible customer service and...
3,329,2187,Essential,handle customer complaints,Administer complaints and negative feedback fr...
4,329,832,Essential,comply with food safety and hygiene,Respect optimal food safety and hygiene during...
5,329,1809,Essential,maintain relationship with customers,Build a lasting and meaningful relationship wi...
6,329,9640,Essential,assist clients with special needs,Aid clients with special needs following relev...
7,329,2861,Essential,provide tourism related information,Give customers relevant information about hist...
8,329,7849,Optional,process reservations,Execute customers' reservations in accordance ...
9,329,434,Optional,distribute local information materials,"Hand out leaflets, maps and tour brochures to ..."


# 5. Occupation similarities

Pre-calculated matrices of occupation similarities can be accessed via the `Similarities()` class.

In [18]:
sim = load_data.Similarities()

In [19]:
# Select the combined similarity measure using 'select_similarity_matrix'
W = sim.select_similarity_matrix('combined')
W.shape

(2942, 2942)

All similarity measures range between 0 (no similarity) and 1 (perfect similarity). We calibrated a threshold similarity value (for the combined similarity measure) that we expect for a *viable* transition. The calibration is described in the report and is performed in the notebook `Transitions_01_Calibrate_viability_threshold.ipynb` in `05_transition_analyses/`. The threshold combined similarity value turned out to be 0.3 for 'minimally viable' transitions, and 0.4 for 'highly viable' ones.

In [20]:
# Check the combined similarity for the transition from 'shop assistant' to 'shelf filler'
W[data.occ_title_to_id('shop assistant'), data.occ_title_to_id('shelf filler')]

0.41204594191789257

For more details, please inspect the `load_data_utils.py`, which features a bit more documentation.